# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

# CNN3D Architecture

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import imageio as imgio
from skimage.transform import resize
import datetime
import os
import warnings
warnings.filterwarnings("ignore")


We set the random seed so that the results don't vary drastically.

In [53]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
#tf.random.set_seed(30)

In [3]:
tf.VERSION

'1.15.0'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
os.getcwd()

'/mnt/disks/user/project/PROJECT/Project_data'

In [54]:
train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/val.csv').readlines())
#train_doc = np.random.permutation(open('C://Users/pchadha/Neural_case_study/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('C://Users/pchadha/Neural_case_study/Project_data/val.csv').readlines())
batch_size = 15

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [18]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            #batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 5%
                    #center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    #width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    #left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    #top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    #image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] Cropped image for model 2 to model 6. Removed it for model 7
                    image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration
                    #image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        #batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 5%
                #center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                #width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                #left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                #top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                #image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                image = resize(image, (100,100,3)) #first model instance and fourth model
                #image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [8]:
#print(len(train_doc)//20)

33


In [39]:
#generator(source_path, train_doc, 50)

<generator object generator at 0x000002507682C048>

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [4]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [55]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers, regularizers
from keras.optimizers import Adam, SGD, RMSprop

#write your model here

Utilizing CNN3D first for model architecture
We will first, build a network with:
- Three convolutional layers having 32, 32 and 64 filters respectively, 
- followed by a max pooling layer, 
- and then `Flatten` the output of the pooling layer to give us a long vector, 
- then add a fully connected `Dense` layer with 128 neurons, and finally
- add a `softmax` layer with 5 neurons

The generic way to build a model in Keras is to instantiate a `Sequential` model and keep adding `keras.layers` to it. We will also use some dropouts.


In [8]:
help(Conv3D)

Help on class Conv3D in module keras.layers.convolutional:

class Conv3D(_Conv)
 |  Conv3D(filters, kernel_size, strides=(1, 1, 1), padding='valid', data_format=None, dilation_rate=(1, 1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
 |  
 |  3D convolution layer (e.g. spatial convolution over volumes).
 |  
 |  This layer creates a convolution kernel that is convolved
 |  with the layer input to produce a tensor of
 |  outputs. If `use_bias` is True,
 |  a bias vector is created and added to the outputs. Finally, if
 |  `activation` is not `None`, it is applied to the outputs as well.
 |  
 |  When using this layer as the first layer in a model,
 |  provide the keyword argument `input_shape`
 |  (tuple of integers, does not include the batch axis),
 |  e.g. `input_shape=(128, 128, 128, 1)` for 128x128x12

In [11]:
help(MaxPooling3D)

Help on class MaxPooling3D in module keras.layers.pooling:

class MaxPooling3D(_Pooling3D)
 |  MaxPooling3D(pool_size=(2, 2, 2), strides=None, padding='valid', data_format=None, **kwargs)
 |  
 |  Max pooling operation for 3D data (spatial or spatio-temporal).
 |  
 |  # Arguments
 |      pool_size: tuple of 3 integers,
 |          factors by which to downscale (dim1, dim2, dim3).
 |          (2, 2, 2) will halve the size of the 3D input in each dimension.
 |      strides: tuple of 3 integers, or None. Strides values.
 |      padding: One of `"valid"` or `"same"` (case-insensitive).
 |      data_format: A string,
 |          one of `channels_last` (default) or `channels_first`.
 |          The ordering of the dimensions in the inputs.
 |          `channels_last` corresponds to inputs with shape
 |          `(batch, spatial_dim1, spatial_dim2, spatial_dim3, channels)`
 |          while `channels_first` corresponds to inputs with shape
 |          `(batch, channels, spatial_dim1, spatial

In [34]:
# model
model1 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model1.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model1.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model1.add(MaxPooling3D(pool_size=(2, 2, 2)))
model1.add(Dropout(0.25))

# Third conv layer
model1.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model1.add(MaxPooling3D(pool_size=(2, 2, 2)))
model1.add(Dropout(0.25))

# flatten and put a fully connected layer
model1.add(Flatten())
model1.add(Dense(128, activation='relu')) # fully connected
model1.add(Dropout(0.5))

# softmax layer
model1.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
#learning rate = 0.01 first model value
learning_rate = 0.002 # second, third and fourth model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 96, 96, 32)    27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 48, 48, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 3, 46, 46, 64)     55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 23, 23, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 23, 23, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [31]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
# Fit or training process
model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0227 18:49:19.868011 139832974251840 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0227 18:49:20.164610 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0227 18:49:20.297953 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0227 18:49:20.430096 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15


W0227 18:49:22.825787 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0227 18:49:22.828643 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0227 18:49:25.215645 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0227 18:49:25.216946 139832974251840 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 70s 2s/step - loss: 1.7085 - categorical_accuracy: 0.2177 - val_loss: 1.4390 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00001-1.72119-0.21569-1.43902-0.24000.h5
Epoch 2/30
45/45 [==============================] - 82s 2s/step - loss: 1.4626 - categorical_accuracy: 0.3246 - val_loss: 1.2958 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00002-1.45844-0.33032-1.29579-0.44000.h5
Epoch 3/30
45/45 [==============================] - 71s 2s/step - loss: 1.3504 - categorical_accuracy: 0.4074 - val_loss: 1.2456 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00003-1.35062-0.40875-1.24558-0.53000.h5
Epoch 4/30
45/45 [==============================] - 82s 2s/step - loss: 1.2073 - categorical_accuracy: 0.5081 - val_loss: 1.1552 - val_categorical_accuracy: 0.470

45/45 [==============================] - 84s 2s/step - loss: 0.1705 - categorical_accuracy: 0.9437 - val_loss: 1.1796 - val_categorical_accuracy: 0.6200

Epoch 00028: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00028-0.17256-0.94268-1.17960-0.62000.h5
Epoch 29/30
45/45 [==============================] - 82s 2s/step - loss: 0.1718 - categorical_accuracy: 0.9453 - val_loss: 1.1796 - val_categorical_accuracy: 0.6200

Epoch 00029: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00029-0.16645-0.95023-1.17960-0.62000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.0240001069306004e-10.
Epoch 30/30
45/45 [==============================] - 67s 1s/step - loss: 0.1957 - categorical_accuracy: 0.9392 - val_loss: 1.1796 - val_categorical_accuracy: 0.6200

Epoch 00030: saving model to pras_model_init_2020-02-2718_48_37.503224/model-00030-0.19800-0.93816-1.17960-0.62000.h5


Selected last 15 images for first trial and resized the image to 100x100.
Tried with batch size from 50 to 15 and epoch size set to 40. Larger the batch size, its generally better however, was getting 'kernal has died' issues which was not resolved even after getting container machine tensorflow updated to version 1.5.
In the end, batch size 15 resolved the issue. Epoch size was kept to be 30.
It was observed that majority of loss reduction was done by epoch 11-12. After that learning rate was being reduced regularly due to no loss reduction for 2 consecutive epochs (original settings)
Will check the accuracy at the end and play with hyper parameters if its not high

In [23]:
# Checking the accuracy
print(model1.history.history.keys())
#print(model1.history.keys())

dict_keys(['val_loss', 'val_categorical_accuracy', 'loss', 'categorical_accuracy', 'lr'])


In [26]:
print(model1.history.history['categorical_accuracy'])

[0.2156862815108774, 0.33031675324046234, 0.4087481276549365, 0.5052790500189923, 0.5444947390804463, 0.6485671353825616, 0.7058823697976937, 0.7692307829856873, 0.761689305305481, 0.8627451062202454, 0.8672699928283691, 0.9095022678375244, 0.9472096562385559, 0.9276018142700195, 0.9260935187339783, 0.9562594294548035, 0.9291101098060608, 0.9457013607025146, 0.9396681785583496, 0.9230769276618958, 0.9351432919502258, 0.9351432919502258, 0.9487179517745972, 0.930618405342102, 0.953242838382721, 0.9472096562385559, 0.9457013607025146, 0.9426847696304321, 0.9502262473106384, 0.9381598830223083]


In [27]:
print(model1.history.history['val_categorical_accuracy'])

[0.24000000730156898, 0.4400000125169754, 0.530000014603138, 0.47000001519918444, 0.5500000104308128, 0.5800000131130219, 0.6100000113248825, 0.5500000149011612, 0.6300000175833702, 0.5900000125169754, 0.6200000137090683, 0.6200000137090683, 0.600000011920929, 0.6300000131130219, 0.6300000190734864, 0.6300000146031379, 0.6200000092387199, 0.6200000181794166, 0.6200000151991845, 0.6200000181794166, 0.6200000226497651, 0.6200000151991845, 0.6200000181794166, 0.6200000226497651, 0.6200000137090683, 0.6200000137090683, 0.6200000151991845, 0.6200000151991845, 0.620000010728836, 0.620000010728836]


We can see that there's lot of difference between training data accuracy and validation data accuracy, training data accuracy being 94%, while validation data, accuracy is just 62%. This is clearly case of overfitting. Therefore, will choose different set of hyper parameter values, and will also select alternate 15 images, instead of last 15.
Will resize the image to 64x64 as well after cropping it by 15 % (centered cropping) using a crop function
Also, will add batch normalization to the two hidden layer network

Second model iteration


In [53]:
# model2
model2 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model2.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model2.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))
model2.add(Dropout(0.25))

# Third conv layer
model2.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling3D(pool_size=(2, 2, 2)))
model2.add(Dropout(0.25))

# flatten and put a fully connected layer
model2.add(Flatten())
model2.add(Dense(128, activation='relu')) # fully connected
model2.add(Dropout(0.5))

# softmax layer
model2.add(Dense(num_classes, activation='softmax'))


In [54]:
learning_rate = 0.002 # second model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 13, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 11, 60, 60, 32)    27680     
_________________________________________________________________
batch_normalization_3 (Batch (None, 11, 60, 60, 32)    128       
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 5, 30, 30, 32)     0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 5, 30, 30, 32)     0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 3, 28, 28, 64)     55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 3, 28, 28, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [55]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [56]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [57]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [58]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
# Fit or training process
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 133s 3s/step - loss: 11.5531 - categorical_accuracy: 0.2235 - val_loss: 12.9008 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to pras_model_init_2020-02-2721_08_51.128419/model-00001-11.66245-0.21569-12.90077-0.20000.h5
Epoch 2/30
45/45 [==============================] - 64s 1s/step - loss: 12.0238 - categorical_accuracy: 0.2312 - val_loss: 10.3598 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to pras_model_init_2020-02-2721_08_51.128419/model-00002-11.95130-0.23529-10.35982-0.35000.h5
Epoch 3/30
45/45 [==============================] - 53s 1s/step - loss: 11.8746 - categorical_accuracy: 0.2505 - val_loss: 13.2706 - val_categorical_accuracy: 0.1700

Epoch 00003: saving model to pras_model_init_2020-02-2721_08_51.128419/model-00003-11.79949-

KeyboardInterrupt: 

Model 2 was not showing improvement, in fact it was showing worse performance
Will now keep the image size as 64x64 ony, but will make dense layer more dense, i.e., increase the number of neurons in this layer

Model3

In [65]:
# model3
model3 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model3.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model3.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(Dropout(0.25))

# Third conv layer
model3.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model3.add(BatchNormalization())
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(Dropout(0.25))

# flatten and put a fully connected layer
model3.add(Flatten())
model3.add(Dense(256, activation='relu')) # fully connected
model3.add(Dropout(0.5))

# softmax layer
model3.add(Dense(num_classes, activation='softmax'))


In [66]:
learning_rate = 0.001 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 13, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 11, 60, 60, 32)    27680     
_________________________________________________________________
batch_normalization_7 (Batch (None, 11, 60, 60, 32)    128       
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 5, 30, 30, 32)     0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 5, 30, 30, 32)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 3, 28, 28, 64)     55360     
_________________________________________________________________
batch_normalization_8 (Batch (None, 3, 28, 28, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [67]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [68]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [69]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [70]:
import warnings
warnings.filterwarnings("ignore")

In [71]:
# Fit or training process
model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 69s 2s/step - loss: 10.3764 - categorical_accuracy: 0.2534 - val_loss: 12.1830 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00001-10.36417-0.25792-12.18305-0.23000.h5
Epoch 2/30
45/45 [==============================] - 65s 1s/step - loss: 11.4839 - categorical_accuracy: 0.2577 - val_loss: 12.5671 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00002-11.49696-0.25641-12.56706-0.20000.h5
Epoch 3/30
45/45 [==============================] - 68s 2s/step - loss: 12.6661 - categorical_accuracy: 0.2001 - val_loss: 13.6516 - val_categorical_accuracy: 0.1400

Epoch 00003: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00003-12.60497-0

45/45 [==============================] - 69s 2s/step - loss: 11.5686 - categorical_accuracy: 0.2431 - val_loss: 12.4648 - val_categorical_accuracy: 0.2200

Epoch 00026: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00026-11.48806-0.24736-12.46480-0.22000.h5
Epoch 27/30
45/45 [==============================] - 48s 1s/step - loss: 11.7690 - categorical_accuracy: 0.2431 - val_loss: 12.3020 - val_categorical_accuracy: 0.2300

Epoch 00027: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00027-11.69204-0.24736-12.30201-0.23000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 8.192000897078167e-13.
Epoch 28/30
45/45 [==============================] - 66s 1s/step - loss: 11.4917 - categorical_accuracy: 0.2576 - val_loss: 12.4709 - val_categorical_accuracy: 0.2200

Epoch 00028: saving model to pras_model_init_2020-02-2721_33_38.963072/model-00028-11.59996-0.25038-12.47095-0.22000.h5
Epoch 29/30
45/45 [==============================] - 45s 1s/step - 

This model has again, shown worse performance. Will now keep the image size as 100x100, select the original last 15 images instead of alternate images

Model4

In [90]:
# model4
model4 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model4.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model4.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model4.add(Dropout(0.25))

# Third conv layer
model4.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model4.add(BatchNormalization())
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model4.add(Dropout(0.25))

# flatten and put a fully connected layer
model4.add(Flatten())
model4.add(Dense(256, activation='relu')) # fully connected
model4.add(Dropout(0.5))

# softmax layer
model4.add(Dense(num_classes, activation='softmax'))


In [91]:
learning_rate = 0.001 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 13, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 11, 96, 96, 32)    27680     
_________________________________________________________________
batch_normalization_11 (Batc (None, 11, 96, 96, 32)    128       
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 5, 48, 48, 32)     0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 3, 46, 46, 64)     55360     
_________________________________________________________________
batch_normalization_12 (Batc (None, 3, 46, 46, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [92]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [93]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [94]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [95]:
import warnings
warnings.filterwarnings("ignore")

In [96]:
# Fit or training process
model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 66s 1s/step - loss: 12.2390 - categorical_accuracy: 0.2075 - val_loss: 12.0436 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00001-12.17030-0.21116-12.04365-0.25000.h5
Epoch 2/30
45/45 [==============================] - 76s 2s/step - loss: 12.0728 - categorical_accuracy: 0.2489 - val_loss: 10.4768 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00002-12.09630-0.24736-10.47676-0.35000.h5
Epoch 3/30
45/45 [==============================] - 63s 1s/step - loss: 11.9969 - categorical_accuracy: 0.2503 - val_loss: 11.7859 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00003-12.01909-0

Epoch 26/30
45/45 [==============================] - 60s 1s/step - loss: 11.7507 - categorical_accuracy: 0.2683 - val_loss: 11.1215 - val_categorical_accuracy: 0.3100

Epoch 00026: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00026-11.67345-0.27300-11.12149-0.31000.h5

Epoch 00026: ReduceLROnPlateau reducing learning rate to 4.096000622011431e-12.
Epoch 27/30
45/45 [==============================] - 80s 2s/step - loss: 11.1828 - categorical_accuracy: 0.2991 - val_loss: 11.1617 - val_categorical_accuracy: 0.3000

Epoch 00027: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00027-11.28559-0.29261-11.16168-0.30000.h5
Epoch 28/30
45/45 [==============================] - 80s 2s/step - loss: 11.6759 - categorical_accuracy: 0.2727 - val_loss: 11.4438 - val_categorical_accuracy: 0.2900

Epoch 00028: saving model to pras_model_init_2020-02-2722_45_38.179634/model-00028-11.59724-0.27753-11.44385-0.29000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate 

This model has again, shown worse performance. Will now keep the image size as 100x100, select the original last 15 images instead of alternate images. Also, was not croppig the image correctly, so corrected the mistake. 
The issue with cropping was that wrong scaling factor (0.05) instead of 0.95 was being used. Will keep higher learning rate as well for model 5 iteration  

Model5 Iteration

In [7]:
# model5
model5 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model5.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model5.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(Dropout(0.25))

# Third conv layer
model5.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model5.add(BatchNormalization())
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(Dropout(0.25))

# flatten and put a fully connected layer
model5.add(Flatten())
model5.add(Dense(128, activation='relu')) # fully connected
model5.add(Dropout(0.5))

# softmax layer
model5.add(Dense(num_classes, activation='softmax'))


W0228 19:36:58.237448 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0228 19:36:58.240003 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0228 19:36:58.399692 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0228 19:36:58.849140 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

In [8]:
learning_rate = 0.005 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

W0228 19:37:06.866137 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0228 19:37:06.873594 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 96, 96, 32)    27680     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 96, 96, 32)    128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 48, 48, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 3, 46, 46, 64)     55360     
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 46, 46, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Fit or training process
model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0228 19:37:57.933302 140257245230912 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0228 19:37:58.575947 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0228 19:37:58.750530 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0228 19:37:58.949256 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15


W0228 19:38:04.189810 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0228 19:38:04.191336 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0228 19:38:10.775106 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0228 19:38:10.778711 140257245230912 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 189s 4s/step - loss: 12.2284 - categorical_accuracy: 0.2134 - val_loss: 13.3668 - val_categorical_accuracy: 0.1700

Epoch 00001: saving model to pras_model_init_2020-02-2819_36_01.751728/model-00001-12.24341-0.21719-13.36685-0.17000.h5
Epoch 2/30
45/45 [==============================] - 87s 2s/step - loss: 11.6694 - categorical_accuracy: 0.2711 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to pras_model_init_2020-02-2819_36_01.751728/model-00002-11.68577-0.26998-12.73330-0.21000.h5
Epoch 3/30
45/45 [==============================] - 91s 2s/step - loss: 10.9017 - categorical_accuracy: 0.3213 - val_loss: 13.3807 - val_categorical_accuracy: 0.1700

Epoch 00003: saving model to pras_model_init_2020-02-2819_36_01.751728/model-00003-10.99957-0.31523-13.38067-0.17000.h5
Epoch 4/30
45/45 [==============================] - 92s 2s/step - loss: 12.1925 - categorical_accuracy: 0.2400 - val_loss: 12.4109 - val_categorical_


Epoch 00026: ReduceLROnPlateau reducing learning rate to 5.11999953545228e-10.
Epoch 27/30
45/45 [==============================] - 73s 2s/step - loss: 12.6635 - categorical_accuracy: 0.2119 - val_loss: 13.0557 - val_categorical_accuracy: 0.1900

Epoch 00027: saving model to pras_model_init_2020-02-2819_36_01.751728/model-00027-12.60238-0.21569-13.05566-0.19000.h5


RuntimeError: Unable to flush file's cached information (file write failed: time = Fri Feb 28 20:14:33 2020
, filename = 'pras_model_init_2020-02-2819_36_01.751728/model-00027-12.60238-0.21569-13.05566-0.19000.h5', file descriptor = 50, errno = 28, error message = 'No space left on device', buf = 0x5577a4386dc0, total write size = 6144, bytes this sub-write = 6144, bytes actually written = 18446744073709551615, offset = 4096)

Model again is performing worse. The changes to selection of images, cropping of images seem to be having adverse effect on performance. Will therefore, first revert to original set of images and then remove cropping if model continues performing poorly  

Model6 Iteration

In [21]:
# model6
model6 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model6.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model6.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model6.add(BatchNormalization())
model6.add(MaxPooling3D(pool_size=(2, 2, 2)))
model6.add(Dropout(0.25))

# Third conv layer
model6.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model6.add(BatchNormalization())
model6.add(MaxPooling3D(pool_size=(2, 2, 2)))
model6.add(Dropout(0.25))

# flatten and put a fully connected layer
model6.add(Flatten())
model6.add(Dense(128, activation='relu')) # fully connected
model6.add(Dropout(0.5))

# softmax layer
model6.add(Dense(num_classes, activation='softmax'))


In [22]:
learning_rate = 0.003 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 13, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 11, 96, 96, 32)    27680     
_________________________________________________________________
batch_normalization_3 (Batch (None, 11, 96, 96, 32)    128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 48, 48, 32)     0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 48, 48, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 3, 46, 46, 64)     55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 3, 46, 46, 64)     256       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
# Fit or training process
model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 153s 3s/step - loss: 12.5839 - categorical_accuracy: 0.1882 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to pras_model_init_2020-02-2820_28_42.834158/model-00001-12.52133-0.19155-12.57212-0.22000.h5
Epoch 2/30
45/45 [==============================] - 79s 2s/step - loss: 12.2306 - categorical_accuracy: 0.2371 - val_loss: 12.1885 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to pras_model_init_2020-02-2820_28_42.834158/model-00002-12.16181-0.24133-12.18846-0.24000.h5
Epoch 3/30
45/45 [==============================] - 60s 1s/step - loss: 11.5927 - categorical_accuracy: 0.2741 - val_loss: 11.7662 - val_categorical_accuracy: 0.2700

Epoch 00003: saving model to pras_model_init_2020-02-2820_28_42.834158/model-00003-11.60771-

KeyboardInterrupt: 

Stopped the model iteration as its not performing well. Selecting the original set of images has not helped much. Will remove the cropping now as well. This will make parameters the same as first model. Therefore, now will add one more hidden layer to improve validation score that we got in first model

Model7 Iteration

In [10]:
# model7
model7 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 15 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model7.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model7.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model7.add(BatchNormalization())
#model7.add(MaxPooling3D(pool_size=(2, 2, 2)))
model7.add(Dropout(0.25))

# Third conv layer
model7.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model7.add(BatchNormalization())
model7.add(MaxPooling3D(pool_size=(2, 2, 2)))
model7.add(Dropout(0.25))

# Fourth conv layer
model7.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model7.add(BatchNormalization())
model7.add(MaxPooling3D(pool_size=(2, 2, 2)))
model7.add(Dropout(0.25))

# flatten and put a fully connected layer
model7.add(Flatten())
model7.add(Dense(128, activation='relu')) # fully connected
model7.add(Dropout(0.5))

# softmax layer
model7.add(Dense(num_classes, activation='softmax'))


In [11]:
learning_rate = 0.003 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model7.summary())

W0228 21:09:10.384916 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 13, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 11, 96, 96, 32)    27680     
_________________________________________________________________
batch_normalization_3 (Batch (None, 11, 96, 96, 32)    128       
_________________________________________________________________
dropout_3 (Dropout)          (None, 11, 96, 96, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 9, 94, 94, 64)     55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 9, 94, 94, 64)     256       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 4, 47, 47, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
# Fit or training process
model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0228 21:10:25.283373 140700025685824 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0228 21:10:25.893525 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0228 21:10:26.120834 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0228 21:10:26.357380 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15


W0228 21:10:28.550758 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0228 21:10:28.552479 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0228 21:10:30.933385 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0228 21:10:30.934820 140700025685824 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 94s 2s/step - loss: 11.0266 - categorical_accuracy: 0.2770 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-02-2821_02_44.214821/model-00001-11.03161-0.27602-12.41093-0.23000.h5
Epoch 2/30
45/45 [==============================] - 75s 2s/step - loss: 10.8545 - categorical_accuracy: 0.3169 - val_loss: 13.0557 - val_categorical_accuracy: 0.1900

Epoch 00002: saving model to pras_model_init_2020-02-2821_02_44.214821/model-00002-10.95149-0.31071-13.05566-0.19000.h5
Epoch 3/30
45/45 [==============================] - 80s 2s/step - loss: 10.8740 - categorical_accuracy: 0.3155 - val_loss: 10.9604 - val_categorical_accuracy: 0.3200

Epoch 00003: saving model to pras_model_init_2020-02-2821_02_44.214821/model-00003-10.87627-0.31523-10.96036-0.32000.h5
Epoch 4/30
45/45 [==============================] - 76s 2s/step - loss: 10.6355 - categorical_accuracy: 0.3319 - val_loss: 9.5153 - val_categorical_ac

KeyboardInterrupt: 

Values again not improving. Will add few more images now

Model8 Iteration

In [21]:
# model8
model8 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 18 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model8.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model8.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model8.add(BatchNormalization())
#model8.add(MaxPooling3D(pool_size=(2, 2, 2)))
model8.add(Dropout(0.25))

# Third conv layer
model8.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model8.add(BatchNormalization())
model8.add(MaxPooling3D(pool_size=(2, 2, 2)))
model8.add(Dropout(0.25))

# Fourth conv layer
model8.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model8.add(BatchNormalization())
model8.add(MaxPooling3D(pool_size=(2, 2, 2)))
model8.add(Dropout(0.25))

# flatten and put a fully connected layer
model8.add(Flatten())
model8.add(Dense(128, activation='relu')) # fully connected
model8.add(Dropout(0.5))

# softmax layer
model8.add(Dense(num_classes, activation='softmax'))


In [22]:
learning_rate = 0.001 # Third model value
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model8.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 16, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 14, 96, 96, 32)    27680     
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 96, 96, 32)    128       
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 96, 96, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 12, 94, 94, 64)    55360     
_________________________________________________________________
batch_normalization_7 (Batch (None, 12, 94, 94, 64)    256       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 6, 47, 47, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator` for second model

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
# Fit or training process
model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0228 21:42:09.868390 140117426284352 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0228 21:42:10.490084 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0228 21:42:10.706593 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0228 21:42:10.972047 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
 /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15


W0228 21:42:13.651912 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0228 21:42:13.654470 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0228 21:42:15.924528 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0228 21:42:15.925768 140117426284352 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 132s 3s/step - loss: 11.4206 - categorical_accuracy: 0.2696 - val_loss: 13.0557 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00001-11.43251-0.26848-13.05566-0.19000.h5
Epoch 2/30
45/45 [==============================] - 95s 2s/step - loss: 11.0304 - categorical_accuracy: 0.2992 - val_loss: 13.0557 - val_categorical_accuracy: 0.1900

Epoch 00002: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00002-11.03540-0.29864-13.05566-0.19000.h5
Epoch 3/30
45/45 [==============================] - 90s 2s/step - loss: 10.0612 - categorical_accuracy: 0.3630 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00003-10.04905-0.36350-12.41093-0.23000.h5
Epoch 4/30
45/45 [==============================] - 93s 2s/step - loss: 10.6786 - categorical_accuracy: 0.3261 - val_loss: 12.4109 - val_categorical_

45/45 [==============================] - 95s 2s/step - loss: 9.4522 - categorical_accuracy: 0.4043 - val_loss: 9.6709 - val_categorical_accuracy: 0.4000

Epoch 00028: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00028-9.52444-0.39970-9.67086-0.40000.h5
Epoch 29/30
45/45 [==============================] - 91s 2s/step - loss: 9.0908 - categorical_accuracy: 0.4266 - val_loss: 9.6709 - val_categorical_accuracy: 0.4000

Epoch 00029: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00029-9.15658-0.42232-9.67086-0.40000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 5.1200004236307e-10.
Epoch 30/30
45/45 [==============================] - 92s 2s/step - loss: 9.6686 - categorical_accuracy: 0.3883 - val_loss: 9.6709 - val_categorical_accuracy: 0.4000

Epoch 00030: saving model to pras_model_init_2020-02-2821_40_59.412193/model-00030-9.55447-0.39517-9.67086-0.40000.h5


Model did not perform well when compared to first model with the same additional layer and almost the same parameters as were considered in the first model. Will now try by adding more images and also cropping the image again. Will also keep the same architecture as the validation and train scores were quite similar. Will reduce the epoch to 20 

Model 9

In [32]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            #batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 5%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration
                    #image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        #batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 5%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                image = resize(image, (100,100,3)) #first model instance and fourth model
                #image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [33]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [35]:
# model 9
model9 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 20 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model9.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model9.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model9.add(MaxPooling3D(pool_size=(2, 2, 2)))
model9.add(Dropout(0.25))

# Third conv layer
model9.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model9.add(MaxPooling3D(pool_size=(2, 2, 2)))
model9.add(Dropout(0.25))

# flatten and put a fully connected layer
model9.add(Flatten())
model9.add(Dense(128, activation='relu')) # fully connected
model9.add(Dropout(0.5))

# softmax layer
model9.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [36]:
import warnings
warnings.filterwarnings("ignore")

In [37]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model9.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 18, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 16, 96, 96, 32)    27680     
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 8, 48, 48, 32)     0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 8, 48, 48, 32)     0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 6, 46, 46, 64)     55360     
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 3, 23, 23, 64)     0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 3, 23, 23, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [38]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [39]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [40]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
# Fit or training process
model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 137s 3s/step - loss: 1.7511 - categorical_accuracy: 0.2370 - val_loss: 1.6091 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to pras_model_init_2020-02-2822_42_49.563538/model-00001-1.75358-0.23529-1.60914-0.21000.h5
Epoch 2/20
45/45 [==============================] - 91s 2s/step - loss: 1.6091 - categorical_accuracy: 0.2148 - val_loss: 1.6064 - val_categorical_accuracy: 0.3700

Epoch 00002: saving model to pras_model_init_2020-02-2822_42_49.563538/model-00002-1.60943-0.21267-1.60641-0.37000.h5
Epoch 3/20
45/45 [==============================] - 99s 2s/step - loss: 1.6088 - categorical_accuracy: 0.1882 - val_loss: 1.4747 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to pras_model_init_2020-02-2822_42_49.563538/model-00003-1.60901-0.19155-1.4

Model has performed better. However, needs to be optimized further. will add couple of more images

Model 10

In [35]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            #batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 5%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration
                    #image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        #batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 5%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                image = resize(image, (100,100,3)) #first model instance and fourth model
                #image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [36]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [37]:
# model 10
model10 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 100
img_cols = 100
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model10.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model10.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model10.add(MaxPooling3D(pool_size=(2, 2, 2)))
model10.add(Dropout(0.25))

# Third conv layer
model10.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model10.add(MaxPooling3D(pool_size=(2, 2, 2)))
model10.add(Dropout(0.25))

# flatten and put a fully connected layer
model10.add(Flatten())
model10.add(Dense(128, activation='relu')) # fully connected
model10.add(Dropout(0.5))

# softmax layer
model10.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model10.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model10.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 20, 98, 98, 32)    2624      
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 18, 96, 96, 32)    27680     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 48, 48, 32)     0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 9, 48, 48, 32)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 7, 46, 46, 64)     55360     
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 23, 23, 64)     0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 3, 23, 23, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [43]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
# Fit or training process
model10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
 /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 126s 3s/step - loss: 1.9927 - categorical_accuracy: 0.2043 - val_loss: 1.5216 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-02-2900_19_10.982650/model-00001-2.01209-0.19608-1.52160-0.23000.h5
Epoch 2/20
45/45 [==============================] - 112s 2s/step - loss: 1.5143 - categorical_accuracy: 0.2978 - val_loss: 1.4658 - val_categorical_accuracy: 0.4100

Epoch 00002: saving model to pras_model_init_2020-02-2900_19_10.982650/model-00002-1.51432-0.29713-1.46578-0.41000.h5
Epoch 3/20
45/45 [==============================] - 101s 2s/step - loss: 1.4322 - categorical_accuracy: 0.3764 - val_loss: 1.3663 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to pras_model_init_2020-02-2900_19_10.982650/model-00003-1.42334-0.38311-1

Being similar to model-1, model is performing great in training set but validation scores are quite less. Will keep the number of images the same but will scale the image to 64x64. Will also, make the last layer more dense 

In [5]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 5%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 5%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [7]:
# model 11
model11 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model11.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model11.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model11.add(MaxPooling3D(pool_size=(2, 2, 2)))
model11.add(Dropout(0.25))

# Third conv layer
model11.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model11.add(MaxPooling3D(pool_size=(2, 2, 2)))
model11.add(Dropout(0.25))

# flatten and put a fully connected layer
model11.add(Flatten())
model11.add(Dense(256, activation='relu')) # fully connected
model11.add(Dropout(0.5))

# softmax layer
model11.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


W0229 01:44:59.478920 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0229 01:44:59.482001 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0229 01:44:59.523870 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0229 01:44:59.563205 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model11.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model11.summary())

W0229 01:45:07.634571 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0229 01:45:07.642872 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 18, 60, 60, 32)    27680     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 30, 30, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 30, 30, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 28, 28, 64)     55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 14, 14, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 14, 14, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Fit or training process
model11.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0229 01:45:24.639739 140215828494144 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0229 01:45:24.972927 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0229 01:45:25.117365 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0229 01:45:25.251022 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/20
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15


W0229 01:45:31.139109 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0229 01:45:31.140705 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0229 01:45:37.888105 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0229 01:45:37.889829 140215828494144 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 224s 5s/step - loss: 1.7180 - categorical_accuracy: 0.2028 - val_loss: 1.4360 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-02-2901_44_53.123820/model-00001-1.72903-0.19457-1.43603-0.23000.h5
Epoch 2/20
45/45 [==============================] - 115s 3s/step - loss: 1.4773 - categorical_accuracy: 0.3141 - val_loss: 1.3749 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to pras_model_init_2020-02-2901_44_53.123820/model-00002-1.47796-0.31373-1.37486-0.43000.h5
Epoch 3/20
45/45 [==============================] - 109s 2s/step - loss: 1.3263 - categorical_accuracy: 0.4134 - val_loss: 1.1103 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to pras_model_init_2020-02-2901_44_53.123820/model-00003-1.32965-0.41478-1.11029-0.54000.h5
Epoch 4/20
45/45 [==============================] - 114s 3s/step - loss: 1.1778 - categorical_accuracy: 0.4919 - val_loss: 0.9484 - val_categorical_accuracy: 0

The model above has performed much better with validate data as well and training score is exceptionally high. Still, difference between train and validation data is much more than desired. So will need to be optimized further. Will crop the image by another 5% for this test. Also, will add two dense layer network rather than one 

Model 12

In [46]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.90, image.shape[0] * 0.90 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.90, image.shape[0] * 0.90 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [47]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [48]:
# model 12
model12 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model12.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)
model12.add(BatchNormalization())
# second conv layer
model12.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model12.add(BatchNormalization())
model12.add(MaxPooling3D(pool_size=(2, 2, 2)))
model12.add(Dropout(0.25))

# Third conv layer
model12.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model12.add(BatchNormalization())
model12.add(MaxPooling3D(pool_size=(2, 2, 2)))
model12.add(Dropout(0.25))

# flatten and put a fully connected layer
model12.add(Flatten())
model12.add(Dense(512, activation='relu')) # fully connected
model12.add(Dropout(0.5))
model12.add(Dense(256, activation='relu')) # fully connected
model12.add(Dropout(0.5))

# softmax layer
model12.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [49]:
import warnings
warnings.filterwarnings("ignore")

In [50]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model12.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model12.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 20, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 18, 60, 60, 32)    27680     
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 9, 30, 30, 32)     0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 9, 30, 30, 32)     0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 7, 28, 28, 64)     55360     
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 3, 14, 14, 64)     0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 3, 14, 14, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [51]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [52]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [53]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [54]:
import warnings
warnings.filterwarnings("ignore")

In [55]:
# Fit or training process
model12.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
45/45 [==============================] - 92s 2s/step - loss: 1.7009 - categorical_accuracy: 0.1867 - val_loss: 1.6095 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to pras_model_init_2020-02-2913_22_01.835205/model-00001-1.70247-0.19005-1.60951-0.18000.h5
Epoch 2/20
45/45 [==============================] - 104s 2s/step - loss: 1.6110 - categorical_accuracy: 0.2075 - val_loss: 1.6069 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to pras_model_init_2020-02-2913_22_01.835205/model-00002-1.61090-0.21116-1.60695-0.18000.h5
Epoch 3/20
45/45 [==============================] - 85s 2s/step - loss: 1.6114 - categorical_accuracy: 0.2074 - val_loss: 1.6077 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to pras_model_init_2020-02-2913_22_01.835205/model-00003-1.61131-0.20513-1.6

The model ended up performing poorly. Will also, not use Batch Normalization as it has proven to be counterproductive in all models where I used it so far. Will now keep the cropping size as 5% only. Will have one Dense layer but that of 512, i.e., one denser layer and increase the number of filters  of second conv3D layer to 64. Will use 'l2' kernel regularizer to see if this helps in reducing overfitting 

Model 13

In [59]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [60]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


In [65]:
# model 13
model13 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model13.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model13.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model13.add(MaxPooling3D(pool_size=(2, 2, 2)))
model13.add(Dropout(0.25))

# Third conv layer
model13.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model13.add(MaxPooling3D(pool_size=(2, 2, 2)))
model13.add(Dropout(0.25))

# flatten and put a fully connected layer
model13.add(Flatten())
model13.add(Dense(512, kernel_regularizer = regularizers.l2(0.01), activation='relu')) # fully connected
model13.add(Dropout(0.5))

# softmax layer
model13.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [66]:
import warnings
warnings.filterwarnings("ignore")

In [67]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model13.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model13.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 20, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 18, 60, 60, 64)    55360     
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 9, 30, 30, 64)     0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 9, 30, 30, 64)     0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 7, 28, 28, 64)     110656    
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 3, 14, 14, 64)     0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 3, 14, 14, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [68]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [69]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [70]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [71]:
import warnings
warnings.filterwarnings("ignore")

In [72]:
# Fit or training process
model13.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
45/45 [==============================] - 103s 2s/step - loss: 5.8505 - categorical_accuracy: 0.2133 - val_loss: 2.8211 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to pras_model_init_2020-02-2914_04_03.771783/model-00001-5.90320-0.21116-2.82106-0.21000.h5
Epoch 2/20
45/45 [==============================] - 76s 2s/step - loss: 2.1902 - categorical_accuracy: 0.2088 - val_loss: 1.8539 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to pras_model_init_2020-02-2914_04_03.771783/model-00002-2.19605-0.20664-1.85387-0.21000.h5
Epoch 3/20
45/45 [==============================] - 105s 2s/step - loss: 1.7471 - categorical_accuracy: 0.2030 - val_loss: 1.6776 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to pras_model_init_2020-02-2914_04_03.771783/model-00003-1.74777-0.20664-1.

KeyboardInterrupt: 

Stopped the model execution as again, model performance was detoriating. Will remove the l2-regularization and then run the model now. Will also increase epoch to 30

Model 14

In [29]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(8,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),64,64,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (64,64,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [30]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [31]:
# model 14
model14 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 64
img_cols = 64
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model14.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model14.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model14.add(MaxPooling3D(pool_size=(2, 2, 2)))
model14.add(Dropout(0.25))

# Third conv layer
model14.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model14.add(MaxPooling3D(pool_size=(2, 2, 2)))
model14.add(Dropout(0.30))

# flatten and put a fully connected layer
model14.add(Flatten())
model14.add(Dense(512, activation='relu')) # fully connected
model14.add(Dropout(0.5))

# softmax layer
model14.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model14.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model14.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 20, 62, 62, 32)    2624      
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 18, 60, 60, 64)    55360     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 30, 30, 64)     0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 9, 30, 30, 64)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 7, 28, 28, 64)     110656    
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 14, 14, 64)     0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 3, 14, 14, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [36]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [38]:
# Fit or training process
model14.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train Source path = ; batch size =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
15
45/45 [==============================] - 94s 2s/step - loss: 1.8329 - categorical_accuracy: 0.2429 - val_loss: 1.5796 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-02-2918_40_44.975965/model-00001-1.83897-0.24133-1.57961-0.23000.h5
Epoch 2/30
45/45 [==============================] - 102s 2s/step - loss: 1.5713 - categorical_accuracy: 0.2459 - val_loss: 1.4474 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to pras_model_init_2020-02-2918_40_44.975965/model-00002-1.57439-0.24434-1.44740-0.39000.h5
Epoch 3/30
45/45 [==============================] - 79s 2s/step - loss: 1.4510 - categorical_accuracy: 0.3333 - val_loss: 1.3121 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to pras_model_init_2020-02-2918_40_44.975965/model-00003-1.44745-0.33333-1.3

Exception ignored in: 'h5py._objects.ObjectID.__dealloc__'
Traceback (most recent call last):
  File "h5py/_objects.pyx", line 193, in h5py._objects.ObjectID.__dealloc__
RuntimeError: Problems closing file (file write failed: time = Sat Feb 29 19:19:40 2020
, filename = 'pras_model_init_2020-02-2918_30_51.546803/model-00003-1.55190-0.25189-1.37962-0.37000.h5', file descriptor = 69, errno = 28, error message = 'No space left on device', buf = 0x55d334d037a0, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)


RuntimeError: Unable to flush file's cached information (file write failed: time = Sat Feb 29 19:19:40 2020
, filename = 'pras_model_init_2020-02-2918_40_44.975965/model-00024-0.09852-0.97587-1.11051-0.72000.h5', file descriptor = 70, errno = 28, error message = 'No space left on device', buf = 0x55d3368af190, total write size = 2048, bytes this sub-write = 2048, bytes actually written = 18446744073709551615, offset = 4096)

Model stopped due to insufficient memory error. Again, like the best model achieved so far, training score was well above 90 but validation score was little over 70. Will now scale the image size to 48x48 and see if this helps in reduction of overfitting

Model 15

In [7]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(8,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [8]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [9]:
# model 15
model15 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model15.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model15.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model15.add(MaxPooling3D(pool_size=(2, 2, 2)))
model15.add(Dropout(0.30))

# Third conv layer
model15.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model15.add(MaxPooling3D(pool_size=(2, 2, 2)))
model15.add(Dropout(0.30))

# flatten and put a fully connected layer
model15.add(Flatten())
model15.add(Dense(512, activation='relu')) # fully connected
model15.add(Dropout(0.5))

# softmax layer
model15.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


W0229 20:30:26.400179 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0229 20:30:26.403157 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0229 20:30:26.468965 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0229 20:30:26.520292 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model15.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model15.summary())

W0229 20:31:01.495034 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0229 20:31:01.502064 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 18, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 22, 22, 64)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 22, 22, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 10, 10, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
# Fit or training process
model15.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0229 20:32:34.953036 140647504074560 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0229 20:32:35.369562 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0229 20:32:35.504024 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0229 20:32:35.636791 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
 /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15


W0229 20:32:40.980192 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0229 20:32:40.987170 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0229 20:32:47.392564 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0229 20:32:47.393864 140647504074560 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 246s 5s/step - loss: 1.6665 - categorical_accuracy: 0.2090 - val_loss: 1.4464 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to pras_model_init_2020-02-2920_27_59.611703/model-00001-1.66794-0.21267-1.44639-0.41000.h5
Epoch 2/30
45/45 [==============================] - 96s 2s/step - loss: 1.4302 - categorical_accuracy: 0.3674 - val_loss: 1.3374 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to pras_model_init_2020-02-2920_27_59.611703/model-00002-1.43066-0.36802-1.33737-0.44000.h5
Epoch 3/30
45/45 [==============================] - 71s 2s/step - loss: 1.2922 - categorical_accuracy: 0.4518 - val_loss: 1.1195 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to pras_model_init_2020-02-2920_27_59.611703/model-00003-1.29248-0.44796-1.11947-0.52000.h5
Epoch 4/30
45/45 [==============================] - 95s 2s/step - loss: 1.1303 - categorical_accuracy: 0.5036 - val_loss: 0.9662 - val_categorical_accuracy: 0.61

Scaling the image scale to 48x48 is giving slightly better performance than the best model so far. Now will keep the image size as 48x48 and will deploy two dense layers of size 256 and 128 neurons respectively, instead of single 512 layer. Will also reduce the number of filters of second conv layer to 32

Model 16

In [32]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(8,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [33]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [34]:
# model 16
model16 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model16.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model16.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model16.add(MaxPooling3D(pool_size=(2, 2, 2)))
model16.add(Dropout(0.30))

# Third conv layer
model16.add(Conv3D(32, kernel_size=(3, 3, 3), 
                 activation='relu'))
model16.add(MaxPooling3D(pool_size=(2, 2, 2)))
model16.add(Dropout(0.30))

# flatten and put a fully connected layer
model16.add(Flatten())
model16.add(Dense(256, activation='relu')) # fully connected
model16.add(Dropout(0.5))
model16.add(Dense(128, activation='relu')) # fully connected
model16.add(Dropout(0.5))

# softmax layer
model16.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model16.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model16.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 20, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 18, 44, 44, 32)    27680     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 22, 22, 32)     0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 9, 22, 22, 32)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 7, 20, 20, 32)     27680     
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 10, 10, 32)     0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 3, 10, 10, 32)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [37]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [38]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [39]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [40]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
# Fit or training process
model16.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 102s 2s/step - loss: 1.6250 - categorical_accuracy: 0.2072 - val_loss: 1.5469 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00001-1.62908-0.19910-1.54692-0.26000.h5
Epoch 2/30
45/45 [==============================] - 74s 2s/step - loss: 1.5670 - categorical_accuracy: 0.2621 - val_loss: 1.4515 - val_categorical_accuracy: 0.3600

Epoch 00002: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00002-1.56709-0.25490-1.45149-0.36000.h5
Epoch 3/30
45/45 [==============================] - 96s 2s/step - loss: 1.5124 - categorical_accuracy: 0.3421 - val_loss: 1.3487 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00003-1.49927-0.33635-1.3

45/45 [==============================] - 95s 2s/step - loss: 0.4860 - categorical_accuracy: 0.8044 - val_loss: 0.9103 - val_categorical_accuracy: 0.6300

Epoch 00028: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00028-0.48884-0.80090-0.91029-0.63000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 6.400000529538374e-08.
Epoch 29/30
45/45 [==============================] - 65s 1s/step - loss: 0.5049 - categorical_accuracy: 0.7986 - val_loss: 0.9103 - val_categorical_accuracy: 0.6300

Epoch 00029: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00029-0.50568-0.80090-0.91030-0.63000.h5
Epoch 30/30
45/45 [==============================] - 66s 1s/step - loss: 0.4883 - categorical_accuracy: 0.8120 - val_loss: 0.9103 - val_categorical_accuracy: 0.6300

Epoch 00030: saving model to pras_model_init_2020-02-2921_27_27.589979/model-00030-0.48240-0.82051-0.91027-0.63000.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.2800001059076749e-0

Making model simpler has not helped as both training and validation scores have reduced. Will keep the number of filters of second Conv layer to be 64. Will use the early stopping to stop iteration in case no progress on accuracy is being achieved for 5 epochs   

Model 17

In [5]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(8,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [7]:
# model 17
model17 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 22 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model17.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model17.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model17.add(MaxPooling3D(pool_size=(2, 2, 2)))
model17.add(Dropout(0.30))

# Third conv layer
model17.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model17.add(MaxPooling3D(pool_size=(2, 2, 2)))
model17.add(Dropout(0.30))

# flatten and put a fully connected layer
model17.add(Flatten())
model17.add(Dense(256, activation='relu')) # fully connected
model17.add(Dropout(0.5))
model17.add(Dense(128, activation='relu')) # fully connected
model17.add(Dropout(0.5))

# softmax layer
model17.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


W0301 07:34:02.719241 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0301 07:34:02.722075 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 07:34:02.781112 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 07:34:02.826275 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model17.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model17.summary())

W0301 07:34:22.952633 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0301 07:34:22.960447 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 18, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 22, 22, 64)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 22, 22, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 10, 10, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=5)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Fit or training process
model17.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0301 07:35:25.430218 140649066219328 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0301 07:35:25.786520 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0301 07:35:25.945378 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0301 07:35:26.110173 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
15


W0301 07:35:32.042979 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0301 07:35:32.044333 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0301 07:35:38.854555 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0301 07:35:38.856267 140649066219328 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 214s 5s/step - loss: 1.6619 - categorical_accuracy: 0.1925 - val_loss: 1.5694 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-03-0107_33_59.031243/model-00001-1.66443-0.19005-1.56942-0.23000.h5
Epoch 2/30
45/45 [==============================] - 102s 2s/step - loss: 1.5861 - categorical_accuracy: 0.2059 - val_loss: 1.4492 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to pras_model_init_2020-03-0107_33_59.031243/model-00002-1.58848-0.20362-1.44920-0.24000.h5
Epoch 3/30
45/45 [==============================] - 105s 2s/step - loss: 1.5221 - categorical_accuracy: 0.2831 - val_loss: 1.3490 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to pras_model_init_2020-03-0107_33_59.031243/model-00003-1.51615-0.28808-1.34899-0.40000.h5
Epoch 4/30
45/45 [==============================] - 106s 2s/step - loss: 1.4603 - categorical_accuracy: 0.3438 - val_loss: 1.3145 - val_categorical_accuracy: 0

Model has not improved the validation score. Will start adding more images, starting from one more, and again, see if it makes difference

Model 18

In [20]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(7,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [21]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [22]:
# model 18
model18 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 23 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model18.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model18.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model18.add(MaxPooling3D(pool_size=(2, 2, 2)))
model18.add(Dropout(0.30))

# Third conv layer
model18.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model18.add(MaxPooling3D(pool_size=(2, 2, 2)))
model18.add(Dropout(0.30))

# flatten and put a fully connected layer
model18.add(Flatten())
model18.add(Dense(256, activation='relu')) # fully connected
model18.add(Dropout(0.50))
model18.add(Dense(256, activation='relu')) # fully connected
model18.add(Dropout(0.50))

# softmax layer
model18.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [23]:
import warnings
warnings.filterwarnings("ignore")

In [24]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model18.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model18.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 21, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 19, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 22, 22, 64)     0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 22, 22, 64)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 7, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 3, 10, 10, 64)     0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 3, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [25]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [26]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=5)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
# Fit or training process
model18.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15

Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 201s 4s/step - loss: 1.6654 - categorical_accuracy: 0.1985 - val_loss: 1.6004 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to pras_model_init_2020-03-0108_53_41.874780/model-00001-1.66735-0.19608-1.60044-0.18000.h5
Epoch 2/30
45/45 [==============================] - 76s 2s/step - loss: 1.5803 - categorical_accuracy: 0.2429 - val_loss: 1.4525 - val_categorical_accuracy: 0.3100

Epoch 00002: saving model to pras_model_init_2020-03-0108_53_41.874780/model-00002-1.58110-0.24133-1.45255-0.31000.h5
Epoch 3/30
45/45 [==============================] - 83s 2s/step - loss: 1.5354 - categorical_accuracy: 0.2771 - val_loss: 1.5266 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to pras_model_init_2020-03-0108_53_41.874780/model-00003-1.53424-0.28205-1.5

Model has performed better. Will therefore, increase number of image by one more. 

Model 19

In [31]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(6,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [32]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [33]:
# model 19
model19 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 24 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model19.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model19.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model19.add(MaxPooling3D(pool_size=(2, 2, 2)))
model19.add(Dropout(0.30))

# Third conv layer
model19.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model19.add(MaxPooling3D(pool_size=(2, 2, 2)))
model19.add(Dropout(0.30))

# flatten and put a fully connected layer
model19.add(Flatten())
model19.add(Dense(256, activation='relu')) # fully connected
model19.add(Dropout(0.50))
model19.add(Dense(256, activation='relu')) # fully connected
model19.add(Dropout(0.50))

# softmax layer
model19.add(Dense(num_classes, activation='softmax'))

# model summary
#model1.summary()


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [34]:
import warnings
warnings.filterwarnings("ignore")

In [35]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model19.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model19.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 22, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 20, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 22, 22, 64)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 22, 22, 64)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 8, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 4, 10, 10, 64)     0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 4, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [36]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [37]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=5)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [38]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [39]:
import warnings
warnings.filterwarnings("ignore")

In [40]:
# Fit or training process
model19.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
 /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 135s 3s/step - loss: 1.6753 - categorical_accuracy: 0.1986 - val_loss: 1.6081 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to pras_model_init_2020-03-0109_28_44.914071/model-00001-1.67633-0.20211-1.60809-0.21000.h5
Epoch 2/30
45/45 [==============================] - 108s 2s/step - loss: 1.5990 - categorical_accuracy: 0.1971 - val_loss: 1.4960 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to pras_model_init_2020-03-0109_28_44.914071/model-00002-1.59881-0.20060-1.49600-0.23000.h5
Epoch 3/30
45/45 [==============================] - 110s 2s/step - loss: 1.5276 - categorical_accuracy: 0.3213 - val_loss: 1.3868 - val_categorical_accuracy: 0.4200

Epoch 00003: saving model to pras_model_init_2020-03-0109_28_44.914071/model-00003-1.53209-0.31523-1

Model performance did not improve. Will now return to best performing model architecture so far and try that with one more image and slightly higher dropout parameter. 

Model-20

In [5]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(7,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [7]:
# model 20
model20 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 23 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model20.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model20.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model20.add(MaxPooling3D(pool_size=(2, 2, 2)))
model20.add(Dropout(0.35))

# Third conv layer
model20.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model20.add(MaxPooling3D(pool_size=(2, 2, 2)))
model20.add(Dropout(0.35))

# flatten and put a fully connected layer
model20.add(Flatten())
model20.add(Dense(512, activation='relu')) # fully connected
model20.add(Dropout(0.5))

# softmax layer
model20.add(Dense(num_classes, activation='softmax'))

# model summary


W0301 19:39:42.487816 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0301 19:39:42.490810 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 19:39:42.614988 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 19:39:42.663658 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model20.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model20.summary())

W0301 19:40:02.061957 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0301 19:40:02.069444 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 21, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 19, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 22, 22, 64)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 22, 22, 64)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 10, 10, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=6)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Fit or training process
model20.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0301 19:40:35.148517 140606473676608 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0301 19:40:35.505953 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0301 19:40:35.641750 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0301 19:40:35.774736 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
 /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15


W0301 19:40:41.950914 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0301 19:40:41.952640 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0301 19:40:48.486605 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0301 19:40:48.488322 140606473676608 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 233s 5s/step - loss: 1.6587 - categorical_accuracy: 0.2148 - val_loss: 1.4763 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to pras_model_init_2020-03-0119_39_31.700400/model-00001-1.66438-0.21267-1.47631-0.24000.h5
Epoch 2/30
45/45 [==============================] - 111s 2s/step - loss: 1.4773 - categorical_accuracy: 0.3157 - val_loss: 1.3562 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to pras_model_init_2020-03-0119_39_31.700400/model-00002-1.47584-0.32127-1.35619-0.39000.h5
Epoch 3/30
45/45 [==============================] - 108s 2s/step - loss: 1.3078 - categorical_accuracy: 0.4488 - val_loss: 1.1563 - val_categorical_accuracy: 0.5100

Epoch 00003: saving model to pras_model_init_2020-03-0119_39_31.700400/model-00003-1.31186-0.44495-1.15628-0.51000.h5
Epoch 4/30
45/45 [==============================] - 123s 3s/step - loss: 1.1363 - categorical_accuracy: 0.5304 - val_loss: 1.1612 - val_categorical_accuracy: 0

Model performance did improve, infact reached higher the score than the best model so far.

Model 21: Increased images by one, i,e, considered images (6-29) and increased dropout value of dense layer to 0.6

In [5]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(6,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [7]:
# model 21
model21 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 24 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model21.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model21.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model21.add(MaxPooling3D(pool_size=(2, 2, 2)))
model21.add(Dropout(0.35))

# Third conv layer
model21.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model21.add(MaxPooling3D(pool_size=(2, 2, 2)))
model21.add(Dropout(0.35))

# flatten and put a fully connected layer
model21.add(Flatten())
model21.add(Dense(512, activation='relu')) # fully connected
model21.add(Dropout(0.6))

# softmax layer
model21.add(Dense(num_classes, activation='softmax'))

# model summary


W0301 21:15:17.879889 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0301 21:15:17.882623 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 21:15:17.956442 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 21:15:18.008820 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model21.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model21.summary())

W0301 21:15:23.037102 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0301 21:15:23.044458 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 22, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 20, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 22, 22, 64)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 22, 22, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 8, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 10, 10, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=7)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
# Fit or training process
model21.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

W0301 21:15:33.510824 139673403180864 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0301 21:15:33.825703 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0301 21:15:33.958622 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0301 21:15:34.082069 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: T

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15


W0301 21:15:40.541683 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0301 21:15:40.544059 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0301 21:15:46.978258 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0301 21:15:46.982310 139673403180864 module_wrapper.py:139] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.

45/45 [==============================] - 236s 5s/step - loss: 1.6720 - categorical_accuracy: 0.2250 - val_loss: 1.4049 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to pras_model_init_2020-03-0121_15_14.502505/model-00001-1.68550-0.21719-1.40486-0.26000.h5
Epoch 2/30
45/45 [==============================] - 116s 3s/step - loss: 1.4494 - categorical_accuracy: 0.3453 - val_loss: 1.3005 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to pras_model_init_2020-03-0121_15_14.502505/model-00002-1.44624-0.35143-1.30051-0.48000.h5
Epoch 3/30
45/45 [==============================] - 113s 3s/step - loss: 1.3466 - categorical_accuracy: 0.4043 - val_loss: 1.2005 - val_categorical_accuracy: 0.4700

Epoch 00003: saving model to pras_model_init_2020-03-0121_15_14.502505/model-00003-1.35110-0.39970-1.20048-0.47000.h5
Epoch 4/30
45/45 [==============================] - 116s 3s/step - loss: 1.2104 - categorical_accuracy: 0.4400 - val_loss: 1.1251 - val_categorical_accuracy: 0

Model performance did not improve, infact detoriated. Model performance did not improve, it detoriated. Will now try some changes to architecture to make it simpler, keeping the number of images same as the best model-20. Will replace single 512 neuron dense layer with two dense layers, one with 256 neurons while the other with 128. Dropout values considered for these layers will be 0.35

Model 22

In [28]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(7,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [29]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [30]:
# model 22
model22 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 23 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model22.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model22.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model22.add(MaxPooling3D(pool_size=(2, 2, 2)))
model22.add(Dropout(0.35))

# Third conv layer
model22.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model22.add(MaxPooling3D(pool_size=(2, 2, 2)))
model22.add(Dropout(0.35))

# flatten and put a fully connected layer
model22.add(Flatten())
model22.add(Dense(256, activation='relu')) # fully connected
model22.add(Dropout(0.35))
model22.add(Dense(128, activation='relu')) # fully connected
model22.add(Dropout(0.35))

# softmax layer
model22.add(Dense(num_classes, activation='softmax'))

# model summary



Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model22.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model22.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 21, 46, 46, 32)    2624      
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 19, 44, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 22, 22, 64)     0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 22, 22, 64)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 7, 20, 20, 64)     110656    
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 3, 10, 10, 64)     0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 3, 10, 10, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [33]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [34]:
model_name = 'pras_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#earlystop = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=9)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
#callbacks_list = [checkpoint, LR, earlystop]
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [36]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
# Fit or training process
model22.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
Source path =  /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 15
Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 15
45/45 [==============================] - 96s 2s/step - loss: 1.6508 - categorical_accuracy: 0.1998 - val_loss: 1.5698 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to pras_model_init_2020-03-0214_28_53.600044/model-00001-1.65390-0.19155-1.56976-0.23000.h5
Epoch 2/30
45/45 [==============================] - 70s 2s/step - loss: 1.5972 - categorical_accuracy: 0.2281 - val_loss: 1.5851 - val_categorical_accuracy: 0.2700

Epoch 00002: saving model to pras_model_init_2020-03-0214_28_53.600044/model-00002-1.59728-0.22624-1.58507-0.27000.h5
Epoch 3/30
45/45 [==============================] - 82s 2s/step - loss: 1.5249 - categorical_accuracy: 0.3127 - val_loss: 1.4581 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to pras_model_init_2020-03-0214_28_53.600044/model-00003-1.52203-0.31825-1.45

KeyboardInterrupt: 

Stopping the iteration as model not giving better results than the best model. Will now consider the model-20 as the best CNN3D model. Will then compare the modeling done by my partner using CNN+RNN and then select the best model accordingly.

# CNN + RNN Architecture

In [ ]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import imageio as imgio
from skimage.transform import resize
import datetime
import os

#We set the random seed so that the results don't vary drastically.

np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
#tf.random.set_seed(30)

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import LSTM,GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.optimizers import Adam, SGD, RMSprop

## Model1

In [ ]:
# Gesture Recognition
#In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.


#In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

os.getcwd()

train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/val.csv').readlines())

batch_size = 15

## Generator

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (100,100,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3))
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (100,100,3))
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels



curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40# choose the number of epochs
print ('# epochs =', num_epochs)

## Model


model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(15,100, 100, 3)))
model.add(TimeDistributed(Conv2D(32, (3,3), 
                                  #strides=(1, 1),
                                   #padding='valid',
                                  activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))

model.add(TimeDistributed(Conv2D(64, (3,3), 
                                  #padding='valid', 
                                   activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), 
                                  padding='valid', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))


model.add(TimeDistributed(Flatten()))

model.add(Dropout(0.5))
model.add(GRU(64, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

#Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

#Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)



model_name = 'cnn_rnn_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

model_name = 'cnn_3d_model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

#The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Model 1 Summary
First iteration:
Image size:100,100,3
Image_index=img_idx = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
No of Images considered per sequence:15
Batch_size=15
epochs=40
model layout:
2 conovlution layers of 32 kernel of size (3,3)and relu activaiton,
maxpooling layer (2,2),2 comvolution layers with 64 kernels of size (3,3) and relu activaiton,
Flatten layer, GRU layer with 64 cells and drop out of 0.5
FInal Dense layer with 5 neurons and Softmax output
No of Parameters:6,025,765
No of Parameters:
Learning rate=0.001
optimiser=Adam
Max train accuracy:99
Max validation accuracy:75

Considered the Image size as (100,100) and last 15 images of video. We achieved maximum train accuracy of 99 and validation accuracy of 75 with around 6 million parameters. In the Next iteration we will change the image size to 48 to reduce the number of parameters.

# Model 2 

In [ ]:
batch_size = 15

## Generator
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [9,10,11,12,13,14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
      
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3))
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (48,48,3))
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


img_idx = [9,10,11,12,13,14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]

len(img_idx)


curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40# choose the number of epochs
print ('# epochs =', num_epochs)

## Model
# to the above model we are adding a additional 2 convolution layers with 64 kernel of 3,3 and max poolling layer 
model1 = Sequential()
model1.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(21,48,48,3)))
model1.add(TimeDistributed(Conv2D(32, (3,3), 
                                  #strides=(1, 1),
                                   #padding='valid',
                                  activation='relu')))
model1.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))


model1.add(TimeDistributed(Conv2D(64, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model1.add(TimeDistributed(Conv2D(64, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model1.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))

model1.add(TimeDistributed(Flatten()))

model1.add(Dropout(0.5))
model1.add(GRU(128, return_sequences=False, dropout=0.5))
model1.add(Dense(5, activation='softmax'))
model1.summary()

#Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

#Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'cnn_rnn_model_iter2_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

#The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=40, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Model 2 Summary
In this iteration we are decreasing the image size to reduce the final number of parameters.
Image size:48,48,3
img_idx = [9,10,11,12,13,14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] # considere more images per video then previous model
No of Images considered per sequence:21
Batch_size=15
epochs=40 
model layout:
2 conovlution layers of 32 kernel of size (3,3)and relu activaiton,
maxpooling layer (2,2),
2 comvolution layers with 64 kernels of size (3,3) and relu activaiton,
Flatten layer, GRU layer with 64 cells and drop out of 0.5
FInal Dense layer with 5 neurons and Softmax output
No of Parameters:2,106,405
Learning rate=0.001
optimiser=Adam
train accuracy:90
validation accuracy:69

In this model there is no significant increase in the validation accuracy even though we have considered more number of images.
In the next model we will change the structure of the network to include more layers.

# Model 3

In [ ]:
batch_size = 15



## Generator
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,29] #selecting every alternate frame  
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),64,64,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (64,64,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),64,64,3))
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (64,64,3))
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,29] 

len(img_idx)


curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

## Model

model2 = Sequential()
model2.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(16,64,64,3)))
model2.add(TimeDistributed(Conv2D(32, (3,3), 
                                  #strides=(1, 1),
                                   #padding='valid',
                                  activation='relu')))
model2.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))

model2.add(TimeDistributed(Conv2D(64, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(Conv2D(64, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))
model2.add(TimeDistributed(Conv2D(128, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(Conv2D(128, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       strides=(1, 1)
                                      )))

model2.add(TimeDistributed(Conv2D(256, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(Conv2D(256, (3,3), 
                                 padding='valid', 
                                 activation='relu')))
model2.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       strides=(1, 1)
                                      )))
model2.add(TimeDistributed(Flatten()))

model2.add(Dropout(0.5))
model2.add(GRU(128, return_sequences=False, dropout=0.5))
model2.add(Dense(5, activation='softmax'))
model2.summary()

#Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

#Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)



model_name = 'cnn_rnn_model_iter3_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

#The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Model 3 Summary:
In this iteration I am adding more convolution layers to reduce the number of parameters at GRU layer and helps in using the  GRU layer with 128 cells
Also instead of using the last images of the video I am using evry alternate image of the each video. 
Image size 
img_idx = [9,10,11,12,13,14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
No of Images considered per sequence:21
Batch_size=15
epochs=20 # reduced number of epochs since there is no considerable change of 20th epoch
model layout:
2 conovlution layers of 32 kernel of size (3,3)and relu activaiton,
maxpooling layer (2,2),
2 comvolution layers with 64 kernels of size (3,3) and relu activaiton,
2 comvolution layers with 128 kernels of size (3,3) and relu activaiton,
2 comvolution layers with 256 kernels of size (3,3) and relu activaiton,
Flatten layer, GRU layer with 64 cells and drop out of 0.5
FInal Dense layer with 5 neurons and Softmax output
No of Parameters:2,107,173
Learning rate=0.001
optimiser=Adam
train accuracy:21
validation accuracy:21
This model has the worst perfomance of the three instead of using more convolution layers and GRU cells.
This Might be due to the less image size and less number of features to GRU layer compared to previous layers


# Model 4:


In [ ]:
batch_size = 15

## Generator

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (100,100,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3))
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (100,100,3))
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


#print(len(train_doc)//50)

#generator(source_path, train_doc, 50)

#Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40# choose the number of epochs
print ('# epochs =', num_epochs)

## Model

model4 = Sequential()
model4.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(15,100, 100, 3)))
model4.add(TimeDistributed(Conv2D(32, (3,3), 
                                  #strides=(1, 1),
                                   #padding='valid',
                                  activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), 
                                       #strides=(1, 1)
                                      )))

model4.add(TimeDistributed(Conv2D(64, (3,3), 
                                  #padding='valid', 
                                   activation='relu')))
model4.add(TimeDistributed(Conv2D(64, (3,3), 
                                  padding='valid', activation='relu')))
model4.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))


model4.add(TimeDistributed(Flatten()))

model4.add(Dropout(0.5))
model4.add(LSTM(64, return_sequences=False, dropout=0.5))
model4.add(Dense(5, activation='softmax'))
model4.summary()

#Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#print (model4.summary())

#Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'cnn_rnn_model_init_iter4' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

#The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#### Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


# Model Summary:
IN this model I tried running model1 by replacing GRU layer with LSTM Layer
Even after replacing the GRU with LSTM we got validation accuracy of 65, but number of parameters has increased 

# Model 5

In [ ]:
batch_size = 15

## Generator
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(8,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels




curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40# choose the number of epochs
print ('# epochs =', num_epochs)


model5 = Sequential()
model5.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(22,48,48, 3)))

model5.add(TimeDistributed(Conv2D(64, (3,3), 
                                  #padding='valid', 
                                   activation='relu')))

model5.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))
model5.add(Dropout(0.30))
model5.add(TimeDistributed(Conv2D(64, (3,3), 
                                  padding='valid', activation='relu')))
model5.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))
model5.add(Dropout(0.30))


model5.add(TimeDistributed(Flatten()))
model5.add(GRU(128, return_sequences=False, dropout=0.5))
model5.add(Dense(256,activation='relu'))
model5.add(Dropout(0.50))
model5.add(Dense(5, activation='softmax'))
model5.summary()

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#print (model4.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'cnn_rnn_model_init_iter5' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#### Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

num_epochs=30

model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Model Summary
THis model we changed reduced the Number of Convolution layers and added more droput layer and Dense layer after GRU layer, It helps in increase of Validation accuracy and more consistent accuracy.
 In the next model we will increase the number of input images and Involve Batch Normalization.ANd increase neurons in Dense layer
 Train Accuracy:93
 Validation Accuracy:73

# Model 6

In [ ]:
batch_size = 15

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(5,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels



curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
#train_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
#val_path = 'C://Users/pchadha/Neural_case_study/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40# choose the number of epochs
print ('# epochs =', num_epochs)

## Model

model6 = Sequential()
model6.add(TimeDistributed(Conv2D(32, (3, 3), 
                                 #strides=(1, 1), 
                                 activation='relu', 
                                 #padding='valid'
                                ), input_shape=(25,48,48, 3)))
model6.add(BatchNormalization())
model6.add(TimeDistributed(Conv2D(64, (3,3), 
                                  #padding='valid', 
                                   activation='relu')))
model6.add(BatchNormalization())
model6.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))
model6.add(Dropout(0.30))
model6.add(TimeDistributed(Conv2D(64, (3,3), 
                                  padding='valid', activation='relu')))
model6.add(BatchNormalization())
model6.add(TimeDistributed(MaxPooling2D((2, 2), 
                                      #strides=(1, 1)
                                      )))
model6.add(Dropout(0.30))


model6.add(TimeDistributed(Flatten()))
model6.add(GRU(128, return_sequences=False, dropout=0.5))
model6.add(Dense(512,activation='relu'))
model6.add(Dropout(0.50))
model6.add(Dense(5, activation='softmax'))
model6.summary()

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

learning_rate = 0.001
optimiser = Adam(lr=learning_rate,clipvalue=1.0)
model6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#print (model4.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'cnn_rnn_model_init_iter6' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

#### Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

num_epochs=30

model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 

                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Model Summary
IN this model I have introduced the Batch Normalization and Increased the Number of Input Images. The validation accuracy didn't improve much 
But I got consistent validation accuracy of 73 across the epochs.
The best accuracy is
Train accuracy: 87
Validation accuracy:73

# Final Decision
Please note the following:
1. CNN3D Model architecture is used by Prashant for this gesture classification exercise
2. CNN + RNN Model architecture is used by Pitchy for this gesture classification exercise
The best performance for CNN3D model is by model-20, i,e., with training accuracy score of 93% and validation score of 78%. 
The best performance by CNN + RNN model is given by model-6, i.e., with training accuracy score of 87 and validation score of 73%.
Now both models need optimiztaion to remove overfitting but considering higher validation score as well as much lower training parameters and computational time required by CNN3D architecture, we are considering CNN3D as main model for classification of video for these sets of gestures.  

## Final model, Model-20 (CNN3D) architecture and generator function for reference

In [5]:
#source_path
#folder_list
#source_path = 'C://Users/pchadha/Neural_case_study/Project_data/train'
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(7,30)] #Selecting second half of images as the differentiating actions within video would most likely be represented by these images  
    #img_idx = [x for x in range(12,30)] # Selecting three more images apart from second 15
    #img_idx = [x for x in range(0,30,2)] #selected for second and third model and fifth model
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,len(img_idx),100,100,3))# x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,len(img_idx),48,48,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imgio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    # Cropping the image for second model analysis by 10%
                    center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                    width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                    image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:] 
                    #image = resize(image, (100,100,3)) # first model instance and fourth model
                    # resizing image to 64x64 for second  and third iteration. Also for model-11
                    image = resize(image, (48,48,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #print(batch_labels.shape(0))
                #print(image.shape(0))
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        # write the code for the remaining data points which are left after full batches
        diff = len(folder_list) - (num_batches*batch_size)
        last_batch_size = diff
        #batch_data = np.zeros((last_batch_size,len(img_idx),100,100,3)) # first model instance and fourth model
        batch_data = np.zeros((last_batch_size,len(img_idx),48,48,3)) #second and third model instance, model 11 as well
        batch_labels = np.zeros((last_batch_size,5))
        for fd in range(diff):
            imgs = os.listdir(source_path+'/'+ t[fd + (num_batches*batch_size)].split(';')[0])
            for idx,item in enumerate(img_idx):
                image = imgio.imread(source_path+'/'+ t[fd + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                # Cropping the image for second model analysis by 10%
                center_x, center_y = image.shape[1] / 2, image.shape[0] / 2
                width_scaled, height_scaled = image.shape[1] * 0.95, image.shape[0] * 0.95 # Corrected the scaling factor for model 5 iteration
                left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
                top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
                image = image[int(top_y):int(bottom_y), int(left_x):int(right_x),:]
                #image = resize(image, (100,100,3)) #first model instance and fourth model
                image = resize(image, (48,48,3)) # second and third model instance
                batch_data[fd,idx,:,:,0] = image[:,:,0]/255 
                batch_data[fd,idx,:,:,1] = image[:,:,1]/255 
                batch_data[fd,idx,:,:,2] = image[:,:,2]/255
            batch_labels[fd, int(t[fd + (num_batches*batch_size)].strip().split(';')[2])] = 1    
        yield batch_data, batch_labels


In [ ]:
# model 20
model20 = Sequential()
#help(Conv3D)
# a keras convolutional layer is called Conv3D

# note that the first layer needs to be told the input shape explicitly

# first conv layer
img_rows = 48
img_cols = 48
imnum = 23 # len(img_idx)
num_classes = 5
input_shape = (imnum, img_rows, img_cols, 3)
model20.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu', 
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model20.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model20.add(MaxPooling3D(pool_size=(2, 2, 2)))
model20.add(Dropout(0.35))

# Third conv layer
model20.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu'))
model20.add(MaxPooling3D(pool_size=(2, 2, 2)))
model20.add(Dropout(0.35))

# flatten and put a fully connected layer
model20.add(Flatten())
model20.add(Dense(512, activation='relu')) # fully connected
model20.add(Dropout(0.5))

# softmax layer
model20.add(Dense(num_classes, activation='softmax'))

# model summary
